In [4]:
#pip install lightgbm

In [2]:
#pip install catboost

# IMPORTATION LIBRAIRIE

In [54]:
import os
import pandas as pd
import warnings
import numpy as np
import gc
import lightgbm as lgb
import joblib
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
warnings.filterwarnings('ignore')

## GET DATA
- manque des ressources, j'ai pris que 1000 lignes de deonnées 

In [55]:
%%time
base_path = "data/"
calendar = pd.read_csv(f"{base_path}calendar.csv")
calendar = calendar.loc[0:1000]

train_eva = pd.read_csv(f"{base_path}sales_train_evaluation.csv")
train_eva = train_eva.loc[0:3000]

sell_prices = pd.read_csv(f"{base_path}sell_prices.csv")
sell_prices = sell_prices.loc[0:3000]

sample_sub = pd.read_csv(f"{base_path}sample_submission.csv")
sample_sub = sample_sub.loc[0:1000]

Wall time: 8.27 s


In [56]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [57]:
train_eva.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [58]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


## PREPROCESSING

In [ ]:
for d in range(1942,1970):
    col = 'd_' + str(d)
    train_eva[col] = 0
    train_eva[col] = train_eva[col].astype(np.int16)

In [33]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

In [34]:
train_eva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Columns: 1975 entries, id to d_1969
dtypes: int16(28), int64(1941), object(6)
memory usage: 44.7+ MB


In [35]:
%%time
print("Downcasting data")
train_eva = downcast(train_eva)
sell_prices = downcast(sell_prices)
calendar = downcast(calendar)

Downcasting data
Wall time: 36.9 s


In [36]:
train_eva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Columns: 1975 entries, id to d_1969
dtypes: category(6), int16(267), int8(1702)
memory usage: 6.6 MB


In [37]:
%%time
print("Melting data")
df = pd.melt(frame=train_eva, 
             id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
             var_name="d", value_name="sold")

Melting data
Wall time: 14.3 s


In [38]:
%%time
print("Merging data")
df = pd.merge(left=df, right=calendar, how="left", on="d")
df = pd.merge(left=df, right=sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")

Merging data
Wall time: 6.92 s


In [39]:
%%time
print("Implement features")
# Chỉ lưu giá trị snap cho từng sản phẩm
df["snap"] = df["snap_CA"] + df["snap_TX"] + df["snap_WI"]
df["snap"] = np.where(df["snap"] >= 1, 1, 0).astype(np.int8)

# Lưu thứ tự ngày ở dạng int
df["d"] = df["d"].str[2:].astype(np.int16)

# Xử lí NaN value cột sell_price
df["sell_price"] = df['sell_price'].fillna(df.groupby('id')['sell_price'].transform('median'))

# Thêm biến có phải cuối tuần hay không
df["weekend"] = np.where(df["wday"] < 3, 1, 0).astype(np.int8)
# Drop các cột không cần thiết
df = df.drop(["date", "weekday", "wm_yr_wk", "event_name_2", "event_type_2", "snap_CA", "snap_TX", "snap_WI"], axis=1)

Implement features
Wall time: 4 s


In [41]:
df["item_id"]

0          HOBBIES_1_001
1          HOBBIES_1_002
2          HOBBIES_1_003
3          HOBBIES_1_004
4          HOBBIES_1_005
               ...      
5908964      FOODS_3_775
5908965      FOODS_3_776
5908966      FOODS_3_777
5908967      FOODS_3_778
5908968      FOODS_3_779
Name: item_id, Length: 5908969, dtype: object

### Les colonnes Label Encoder ont la forme catégorie

In [ ]:
print("Label Encoding")
d_id = dict(zip(df["id"].cat.codes, df["id"]))
df["id"] = df["id"].cat.codes
df["item_id"] = df["item_id"].cat.codes
df["dept_id"] = df["dept_id"].cat.codes
df["cat_id"] = df["cat_id"].cat.codes
df["store_id"] = df["store_id"].cat.codes
df["state_id"] = df["state_id"].cat.codes
df["event_name_1"] = df["event_name_1"].cat.codes
df["event_type_1"] = df["event_type_1"].cat.codes

In [42]:
%%time
print("Mean Encoding")
df["state_mean"] = df.groupby("state_id")["sold"].transform("mean").astype(np.float16)
df["store_mean"] = df.groupby("store_id")["sold"].transform("mean").astype(np.float16)
df["cat_mean"] = df.groupby("cat_id")["sold"].transform("mean").astype(np.float16)
df["dept_mean"] = df.groupby("dept_id")["sold"].transform("mean").astype(np.float16)
df["state_cat_mean"] = df.groupby(["state_id", "cat_id"])["sold"].transform("mean").astype(np.float16)
df["state_dept_mean"] = df.groupby(["state_id", "dept_id"])["sold"].transform("mean").astype(np.float16)
df["store_cat_mean"] = df.groupby(["store_id", "cat_id"])["sold"].transform("mean").astype(np.float16)
df["store_cat_mean"] = df.groupby(["dept_id", "cat_id"])["sold"].transform("mean").astype(np.float16)
df["item_id_mean"] = df.groupby("item_id")["sold"].transform("mean").astype(np.float16)
df["item_state_mean"] = df.groupby(["item_id", "state_id"])["sold"].transform("mean").astype(np.float16)
df["item_store_mean"] = df.groupby(["item_id", "store_id"])["sold"].transform("mean").astype(np.float16)

Mean Encoding
Wall time: 5.53 s


In [43]:
%%time
print("Calulating Lags")
lags = [29,30,31,32,33,34,35,40,55,60,65,180]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

Calulating Lags
Wall time: 19.8 s


In [44]:
df = df[df['d']>max(lags)]

In [45]:
df.to_pickle('data.pkl')
del df, calendar, sell_prices, train_eva
gc.collect()

93

## MODELISATION
- j'ai utilisé trois modeles:
    - LGBMRegressor
    - XGBRegressor
    - CatBoostRegressor

In [46]:
def lightgbm_model(X_train, y_train, X_valid, y_valid):
    model = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.3,
        subsample=0.8,
        colsample_bytree=0.8,
        max_depth=8,
        num_leaves=224,
        min_child_weight=300
    )
        
    model.fit(X_train, y_train, 
          eval_set=[(X_train,y_train),(X_valid,y_valid)], 
          verbose=20, 
          early_stopping_rounds=20,
          eval_metric='rmse')
    
    return model

def xgboost_model(X_train, y_train, X_valid, y_valid):
    model = XGBRegressor(
        n_estimators=1000,
        learning_rate=0.3,
        subsample=0.8,
        colsample_bytree=0.8,
        max_depth=8,
        min_child_weight=300
    )

    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=20,
              early_stopping_rounds=20,
              eval_metric='rmse')

    return model

def catboost_model(X_train, y_train, X_valid, y_valid):
    
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.3,
        subsample=0.8,
        colsample_bylevel=0.8,
        depth=8,
        l2_leaf_reg=224,
        min_child_samples=300,
        eval_metric='RMSE'
    )
    
    model.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid,y_valid)],
              verbose=20, early_stopping_rounds=20)
    
    return model

In [47]:
%%time
data = pd.read_pickle('data.pkl')

Wall time: 813 ms


In [48]:
valid = data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]

##### Donnée de test et de validation

### TRAINING 

In [ ]:
%%time
for i in range(10):
    # Forecast cho từng store 
    df = data[data["store_id"] == i]    
    #Create train set
    X_train, y_train = df[df['d']<1914].drop('sold',axis=1), df[df['d']<1914]['sold']
    X_valid, y_valid = df[(df['d']>=1914) & (df['d']<1942)].drop('sold',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sold']
    X_test = df[df["d"] >= 1942].drop("sold", axis=1)
       
    # Create model
    print(f"Train model for store {i}")
    print("--------")
    
    # Tùy chọn sử dụng loại model
    model = lightgbm_model(X_train, y_train, X_valid, y_valid)
    #model = xgboost_model(X_train, y_train, X_valid, y_valid)
    #model = catboost_model(X_train, y_train, X_valid, y_valid)
    
    print("--------")
    print(f"Predicting for store {i}")    
    # Validation predict
    pred_val = model.predict(X_valid)
    valid.loc[X_valid.index, "sold"] = pred_val
    pred_eva = model.predict(X_test)
    test.loc[X_test.index, "sold"] = pred_eva   
    print("--------")
    
    print("Saving model and clear memories")  
    print("--------")
    filename = f'model_store_{i}.pkl'
    # Save model and Clear memmory
    joblib.dump(model, filename)
    del model, X_train, y_train, X_valid, y_valid, X_test
    gc.collect()

### Création de la soumission de fichier

In [50]:
sample_sub = sample_sub[["id"]]
f_col = [f"F{i}" for i in range(1,29)]
f_col.insert(0, "id")

valid["id"] = valid["id"].map(d_id)
valid = valid.pivot(index="id", columns="d", values="sold").reset_index()
valid["id"] = valid["id"].str.replace("evaluation", "validation")
out_val = pd.merge(left=sample_sub[:30490], right=valid, on="id")
out_val.columns=f_col

test["id"] = test["id"].map(d_id)
test = test.pivot(index="id", columns="d", values="sold").reset_index()
out_eva = pd.merge(left=sample_sub[30490:], right=test, on="id")
out_eva.columns=f_col

submit = pd.concat([out_val,out_eva], ignore_index=True)

In [51]:
submit

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,HOUSEHOLD_1_441_CA_1_validation,9,11,2,3,4,6,12,2,7,...,3,8,6,3,7,4,3,4,9,6
997,HOUSEHOLD_1_442_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,1,0,0,0,0
998,HOUSEHOLD_1_443_CA_1_validation,0,2,3,1,1,1,1,2,0,...,4,2,2,2,0,1,3,0,2,2
999,HOUSEHOLD_1_444_CA_1_validation,1,1,0,0,1,1,0,0,1,...,0,0,2,0,1,2,0,0,0,0


In [ ]:
submit.to_csv('submission.csv',index=False)